# Trying to use new TF HMM code with edward:

The version of Edward that we have been using so far uses an old version of TF, which includes Probability as a submodule tf.contrib.Distribution, and this submodule doesn't contain HMM.

Yet the new version of TF Probability (**release 0.5**, and requires TF **1.12**), which is now a standalone module, does contain HMM. The goal of this notebook is to use that and wrap it as an edward Random Variable to do inference.

**Challenge:** The new TFP actually also contains the newest verion of Edward, as Edward2. But they haven't ported everything from Edward to Edward2 yet, in particular not the inference. We probably want to try to use only the new HMM from the new TF and use it in our (old) version of Edward.

In [1]:
import tensorflow as tf
import edward as ed
import numpy as np
import tensorflow.contrib.distributions as tfd

Instructions for updating:
Use the retry module or similar alternatives.


## Porting HMM from new TFP to our version of edward and tf:

In [2]:
from os.path import join, abspath
import sys
sys.path.append(join(abspath('.'), '../utils'))
from tf_hidden_markov_model import HiddenMarkovModel

### Example, from tfp:

In [3]:
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])

Suppose a cold day has a 30% chance of being followed by a hot day and a hot day has a 20% chance of being followed by a cold day.

We can model this as:

In [4]:
transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]])

Suppose additionally that on each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.

We can model this with:

In [5]:
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])

We can combine these distributions into a single week long hidden Markov model with:

In [6]:
model = HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

In [7]:
with tf.Session() as sess:
    # The expected temperatures for each day are given by:
    print(sess.run(model.mean()))  # shape [7], elements approach 9.0
    # The log pdf of a week of temperature 0 is:
    print(sess.run(model.log_prob(tf.zeros(shape=[7]))))

Tensor("HiddenMarkovModel/cond/Merge:0", shape=(?,), dtype=float32)
Tensor("HiddenMarkovModel/mean/Normal/mean/mul:0", shape=(2,), dtype=float32)
[2.9999998 5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]
Tensor("HiddenMarkovModel/cond/Merge:0", shape=(?,), dtype=float32)
Tensor("zeros:0", shape=(7,), dtype=float32)
-20.120832


## Trying to run inference on this example: